### Dogs Breed Descriptor

Esse notebook foi feito para modificar o modelo de classificacao de modo a dar o output de um descriptor com 512 valores, sendo utilizado para a parte final do projeto

by: Crystal Silva Campos <https://github.com/campos537>

### Step 1
importar as bibliotecas

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
# Plot library
import matplotlib.pyplot as plt
# Fast array processing library
import numpy as np
# Deep Learning framework
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
# Library to find the best learning rate
from torch_lr_finder import LRFinder
from autoaugment import ImageNetPolicy, CIFAR10Policy, SVHNPolicy, SubPolicy
import cv2

### Step 2
Nesse passo vou carregar o modelo e setar alguns parametros utilizados pelo mesmo

In [2]:
input_size = (224,224)
device = torch.device("cuda")
model_name = "dog_class_model_resnet50.pt"
model_loaded = torch.load(model_name)
model_loaded.eval()
model_loaded.to("cpu")

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Step 3
Remove as ultimas layers da parte de classificacao para fazer o modelo funcionar retornando um vetor de features com 512 descriptors.

In [3]:
model_loaded.fc = nn.Sequential(nn.Linear(2048, 512))

### Step 4
Salva o modelo no formato ONNX

In [5]:
model_loaded.eval()
model_loaded.to("cpu")
output_onnx_name = "desc_" + model_name[:-3] + ".onnx"
dummy_input = torch.randn(1, 3, input_size[0], input_size[1], device='cpu')
torch.onnx.export(model_loaded, dummy_input, output_onnx_name , verbose=True)

/media/disk1/Repositorios/UnicoID/dogs-project/dogs-env/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


graph(%input.1 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %fc.0.weight : Float(512, 2048, strides=[2048, 1], requires_grad=1, device=cpu),
      %fc.0.bias : Float(512, strides=[1], requires_grad=1, device=cpu),
      %497 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %498 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %500 : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %501 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %503 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %504 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %506 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %507 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %509 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %510 : Float(256, strides=[1], re

### Step 5
Checar o modelo onnx gerado

In [6]:
import onnx
onnx_model = onnx.load(output_onnx_name)
onnx.checker.check_model(onnx_model)
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch-jit-export (
  %input.1[FLOAT, 1x3x224x224]
) initializers (
  %fc.0.weight[FLOAT, 512x2048]
  %fc.0.bias[FLOAT, 512]
  %497[FLOAT, 64x3x7x7]
  %498[FLOAT, 64]
  %500[FLOAT, 64x64x1x1]
  %501[FLOAT, 64]
  %503[FLOAT, 64x64x3x3]
  %504[FLOAT, 64]
  %506[FLOAT, 256x64x1x1]
  %507[FLOAT, 256]
  %509[FLOAT, 256x64x1x1]
  %510[FLOAT, 256]
  %512[FLOAT, 64x256x1x1]
  %513[FLOAT, 64]
  %515[FLOAT, 64x64x3x3]
  %516[FLOAT, 64]
  %518[FLOAT, 256x64x1x1]
  %519[FLOAT, 256]
  %521[FLOAT, 64x256x1x1]
  %522[FLOAT, 64]
  %524[FLOAT, 64x64x3x3]
  %525[FLOAT, 64]
  %527[FLOAT, 256x64x1x1]
  %528[FLOAT, 256]
  %530[FLOAT, 128x256x1x1]
  %531[FLOAT, 128]
  %533[FLOAT, 128x128x3x3]
  %534[FLOAT, 128]
  %536[FLOAT, 512x128x1x1]
  %537[FLOAT, 512]
  %539[FLOAT, 512x256x1x1]
  %540[FLOAT, 512]
  %542[FLOAT, 128x512x1x1]
  %543[FLOAT, 128]
  %545[FLOAT, 128x128x3x3]
  %546[FLOAT, 128]
  %548[FLOAT, 512x128x1x1]
  %549[FLOAT, 512]
  %551[FLOAT, 128x512x1x1]
  %552[FLOAT, 128]
  %554[FLOAT, 128x12

### Step 5
Testar descriptor em imagem usando opencv

In [9]:
img = cv2.imread("chewie1.png")
model = cv2.dnn.readNetFromONNX(output_onnx_name)
blob =  cv2.dnn.blobFromImage(img, 1/255, (224,224), (), True, False)
model.setInput(blob)
out = model.forward()
print(out)

plt.imshow(img)
plt.title('Shitzu')
plt.show()

error: OpenCV(4.5.3) /tmp/pip-req-build-l1r0y34w/opencv/modules/dnn/src/onnx/onnx_importer.cpp:78: error: (-5:Bad argument) Can't read ONNX file: desc_dog_class_model_resnet50.onnx in function 'ONNXImporter'
